# 🌿 Sustain 4.0 BioEngine - Visualização de Usuários e Dados Persistentes

Este notebook permite visualizar todos os usuários cadastrados no sistema Sustain 4.0 BioEngine através da leitura do arquivo `config.yaml`, assim como seus dados persistentes armazenados em arquivos JSON na pasta `data/`.

## Persistência de Dados

O sistema agora implementa persistência completa de dados para:
- Projetos criados pelos usuários
- Preferências personalizadas
- Configurações do sistema

## Usuários cadastrados na streamlit cloud

Para ver o restante dos usuários que se registraram online, será necessário baixar a versão atual do arquivo config.yaml apenas disponível em nuvem do streamlit. Seria necessário criar um script básico e acoplar ao código fonte do app para que o servidor exporte (apenas enquanto logado como adm) o arquivo atual config.yaml na próxima rodagem do app na web.

## 📊 Funcionalidades:

## 1. 📚 Import Required Libraries

In [1]:
# Import necessary libraries
import pandas as pd
import yaml
from datetime import datetime
import os

# Display configurations for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("📚 Bibliotecas importadas com sucesso!")
print(f"📅 Pandas version: {pd.__version__}")
print(f"🕐 Timestamp: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

📚 Bibliotecas importadas com sucesso!
📅 Pandas version: 2.2.3
🕐 Timestamp: 01/08/2025 11:31:30


## 2. 📂 Load YAML Configuration File

In [2]:
# Load YAML configuration file
config_file = 'config.yaml'

try:
    # Check if file exists
    if not os.path.exists(config_file):
        raise FileNotFoundError(f"Arquivo {config_file} não encontrado!")
    
    # Load YAML content
    with open(config_file, 'r', encoding='utf-8') as file:
        config = yaml.safe_load(file)
    
    print(f"✅ Arquivo {config_file} carregado com sucesso!")
    print(f"📊 Estrutura do arquivo:")
    print(f"   - Seções principais: {list(config.keys())}")
    
    # Check if credentials section exists
    if 'credentials' in config and 'usernames' in config['credentials']:
        user_count = len(config['credentials']['usernames'])
        print(f"👥 Total de usuários encontrados: {user_count}")
    else:
        print("⚠️ Seção de credenciais não encontrada!")
        
except FileNotFoundError as e:
    print(f"❌ Erro: {e}")
    config = None
except yaml.YAMLError as e:
    print(f"❌ Erro ao ler YAML: {e}")
    config = None
except Exception as e:
    print(f"❌ Erro inesperado: {e}")
    config = None

✅ Arquivo config.yaml carregado com sucesso!
📊 Estrutura do arquivo:
   - Seções principais: ['cookie', 'credentials', 'preauthorized']
👥 Total de usuários encontrados: 4


## 3. 🔍 Extract User Data

In [3]:
# Extract user data from configuration
if config and 'credentials' in config and 'usernames' in config['credentials']:
    users_data = config['credentials']['usernames']
    
    print("🔍 Extraindo dados dos usuários...")
    print(f"📊 Usuários encontrados: {list(users_data.keys())}")
    
    # Process each user
    user_list = []
    for username, user_info in users_data.items():
        # Extract user information
        user_record = {
            'username': username,
            'name': user_info.get('name', 'N/A'),
            'email': user_info.get('email', 'N/A'),
            'password_hash': user_info.get('password', 'N/A'),
            'user_type': '👑 Admin' if username == 'admin' else '👤 User',
            'status': '🟢 Active'
        }
        user_list.append(user_record)
    
    print(f"✅ Dados extraídos para {len(user_list)} usuários!")
    
    # Display sample of extracted data
    print("\n📋 Amostra dos dados extraídos:")
    for i, user in enumerate(user_list[:3], 1):  # Show first 3 users
        print(f"   {i}. {user['username']} -> {user['name']} ({user['email']})")
        
else:
    print("❌ Não foi possível extrair dados dos usuários!")
    user_list = []

🔍 Extraindo dados dos usuários...
📊 Usuários encontrados: ['admin', 'analista', 'demo', 'pesquisador']
✅ Dados extraídos para 4 usuários!

📋 Amostra dos dados extraídos:
   1. admin -> Administrador (admin@bioengine.com)
   2. analista -> Analista (analista@bioengine.com)
   3. demo -> Demo User (demo@bioengine.com)


## 4. 📊 Create Pandas DataFrame

In [4]:
# Create pandas DataFrame from user data
if user_list:
    # Create DataFrame
    df_users = pd.DataFrame(user_list)
    
    # Add some computed columns
    df_users['password_preview'] = df_users['password_hash'].apply(
        lambda x: x[:20] + '...' if x != 'N/A' else 'N/A'
    )
    df_users['created_date'] = datetime.now().strftime('%d/%m/%Y')
    
    # Reorder columns for better display
    column_order = [
        'username', 
        'name', 
        'email', 
        'user_type', 
        'status', 
        'password_preview',
        'created_date'
    ]
    df_users = df_users[column_order]
    
    print("✅ DataFrame criado com sucesso!")
    print(f"📊 Dimensões: {df_users.shape[0]} linhas x {df_users.shape[1]} colunas")
    print(f"📋 Colunas: {list(df_users.columns)}")
    
else:
    print("❌ Não foi possível criar DataFrame - nenhum dado de usuário disponível!")
    df_users = pd.DataFrame()  # Empty DataFrame

✅ DataFrame criado com sucesso!
📊 Dimensões: 4 linhas x 7 colunas
📋 Colunas: ['username', 'name', 'email', 'user_type', 'status', 'password_preview', 'created_date']


In [ ]:
# Função para carregar dados persistentes do usuário
def load_user_persistent_data(username):
    """Carrega os dados persistentes de um usuário específico"""
    data_file = Path("./data") / f"{username}.json"
    
    if data_file.exists():
        try:
            with open(data_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        except Exception as e:
            print(f"❌ Erro ao carregar dados do usuário {username}: {e}")
            return {}
    else:
        print(f"⚠️ Arquivo de dados persistentes não encontrado para {username}")
        return {}

# Listar todos os arquivos de dados persistentes
data_dir = Path("./data")
if data_dir.exists():
    print("📁 Arquivos de dados persistentes encontrados:")
    for file in data_dir.glob("*.json"):
        print(f"  - {file.name}")
else:
    print("❌ Diretório de dados não encontrado.")

# Carregar dados do usuário 'demo' como exemplo
demo_data = load_user_persistent_data('demo')
print("\n📊 Dados persistentes do usuário 'demo':")
if demo_data:
    print(f"  - Projetos: {len(demo_data.get('projects', []))}")
    print(f"  - Preferências: {demo_data.get('preferences', {})}")
    print(f"  - Última atualização: {demo_data.get('last_update', 'N/A')}")
else:
    print("  - Nenhum dado encontrado para o usuário 'demo'")

## 5. 📈 Display and Analyze User Data

In [5]:
# Display the complete DataFrame
if not df_users.empty:
    print("🌿 SUSTAIN 4.0 BIOENGINE - USUÁRIOS CADASTRADOS")
    print("=" * 60)
    
    # Display the DataFrame
    display(df_users)
    
else:
    print("📭 Nenhum usuário encontrado para exibir.")

🌿 SUSTAIN 4.0 BIOENGINE - USUÁRIOS CADASTRADOS


,username,name,email,user_type,status,password_preview,created_date
0,admin,Administrador,admin@bioengine.com,👑 Admin,🟢 Active,$2b$12$M3cvnwOJjrVwM...,01/08/2025
1,analista,Analista,analista@bioengine.com,👤 User,🟢 Active,$2b$12$hTgy1j3M0jQVF...,01/08/2025
2,demo,Demo User,demo@bioengine.com,👤 User,🟢 Active,$2b$12$.afwGV05MWlrg...,01/08/2025
3,pesquisador,Pesquisador,pesquisador@bioengine.com,👤 User,🟢 Active,$2b$12$qYAd9TDMBeBm8...,01/08/2025


In [6]:
# Detailed analysis and statistics
if not df_users.empty:
    print("\n📊 ANÁLISE DETALHADA DOS DADOS")
    print("=" * 40)
    
    # Basic info
    print(f"📈 Informações Básicas:")
    print(f"   • Total de usuários: {len(df_users)}")
    print(f"   • Colunas disponíveis: {len(df_users.columns)}")
    print(f"   • Tipos de dados:")
    for col in df_users.columns:
        print(f"     - {col}: {df_users[col].dtype}")
    
    # User type distribution
    print(f"\n👥 Distribuição por Tipo de Usuário:")
    user_type_counts = df_users['user_type'].value_counts()
    for user_type, count in user_type_counts.items():
        print(f"   • {user_type}: {count}")
    
    # Email domains analysis
    print(f"\n📧 Análise de Domínios de Email:")
    df_users['email_domain'] = df_users['email'].apply(
        lambda x: x.split('@')[1] if '@' in str(x) else 'N/A'
    )
    domain_counts = df_users['email_domain'].value_counts()
    for domain, count in domain_counts.items():
        print(f"   • {domain}: {count} usuário(s)")
    
    # Status overview
    print(f"\n🟢 Status dos Usuários:")
    status_counts = df_users['status'].value_counts()
    for status, count in status_counts.items():
        print(f"   • {status}: {count}")
    
    print(f"\n🔐 Informações de Segurança:")
    print(f"   • Todas as senhas estão hasheadas com bcrypt")
    print(f"   • Hash médio de caracteres: ~60 caracteres")
    print(f"   • Última atualização: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

else:
    print("📭 Sem dados para análise.")


📊 ANÁLISE DETALHADA DOS DADOS
📈 Informações Básicas:
   • Total de usuários: 4
   • Colunas disponíveis: 7
   • Tipos de dados:
     - username: object
     - name: object
     - email: object
     - user_type: object
     - status: object
     - password_preview: object
     - created_date: object

👥 Distribuição por Tipo de Usuário:
   • 👤 User: 3
   • 👑 Admin: 1

📧 Análise de Domínios de Email:
   • bioengine.com: 4 usuário(s)

🟢 Status dos Usuários:
   • 🟢 Active: 4

🔐 Informações de Segurança:
   • Todas as senhas estão hasheadas com bcrypt
   • Hash médio de caracteres: ~60 caracteres
   • Última atualização: 01/08/2025 11:31:30


## Conclusão

Este notebook permite visualizar e analisar os dados persistentes dos usuários do Sustain 4.0 BioEngine. Com o novo sistema de persistência implementado, os usuários agora podem:

1. **Manter seus projetos entre sessões**: Projetos criados são salvos automaticamente e ficam disponíveis quando o usuário faz login novamente
2. **Personalizar suas preferências**: Configurações como tema, notificações e preferências de visualização são mantidas
3. **Gerenciar configurações do sistema**: Administradores podem configurar o sistema e as alterações persistem

O sistema de persistência é implementado através de:
- Arquivos JSON por usuário na pasta `data/`
- Auto-salvamento periódico durante o uso do aplicativo
- Salvamento explícito em momentos-chave (logout, criação de projeto, alteração de configurações)

Isso melhora significativamente a experiência do usuário e possibilita o desenvolvimento de análises ambientais de longo prazo.